# Trying Different Methods od Reading PDF Files

In [6]:
!pip install PyPDF2

     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     ---------- ---------------------------- 61.4/232.6 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 2.9 MB/s eta 0:00:00


In [5]:
!pip list

Package               Version
--------------------- -----------
accelerate            0.20.3
aiohttp               3.8.4
aiosignal             1.3.1
anyio                 3.7.1
astor                 0.8.1
asttokens             2.2.1
async-timeout         4.0.2
attrdict              2.0.1
attrs                 23.1.0
Babel                 2.12.1
backcall              0.2.0
bce-python-sdk        0.8.87
beautifulsoup4        4.12.2
blinker               1.6.2
cachetools            5.3.1
certifi               2023.5.7
charset-normalizer    3.1.0
click                 8.1.3
colorama              0.4.6
comm                  0.1.3
contourpy             1.1.0
cssselect             1.2.0
cssutils              2.7.1
cycler                0.11.0
Cython                0.29.36
datasets              2.13.1
debugpy               1.6.7
decorator             5.1.1
dill                  0.3.6
et-xmlfile            1.1.0
exceptiongroup        1.1.2
executing             1.2.0
filelock              3.12.2

## Trying PyPDF2 Library

In [7]:
from PyPDF2 import PdfReader
import re

In [10]:
pdf_path = "./Investasi/2-2009-UU-PERTAMBANGAN.pdf"
pdf = PdfReader(open(pdf_path, "rb"))

In [11]:
print("Total number of pages:", len(pdf.pages))

Total number of pages: 87


In [ ]:
text = ""
for page_number in range(len(pdf.pages)): #len(pdf.pages)
    page = pdf.pages[page_number]
    text += page.extract_text()
print(text)

### Conclusion
PyPDF2 is neater in comparison to PyMuPDF

## Trying to Clean Extracted Text

In [15]:
text = re.sub(r'\n', '', text)
text

'd. bahwa . . . UNDANG-UNDANG REPUBLIK INDONESIA  NOMOR  4 TAHUN 2009      TENTANG  PERTAMBANGAN MINERAL DAN BATUBARA   DENGAN RAHMAT TUHAN YANG MAHA ESA     PRESIDEN REPUBLIK INDONESIA,   Menimbang :  a. bahwa  mineral  dan  batubara  yang  te rkandung dalam  wilayah hukum pertambangan Indonesia merupakan kekayaan alam tak terbarukan sebagai karunia Tuhan Yang Maha Esa yang mempunyai peranan penting dalam memenuhi hajat hidup orang banyak, karena itu pengelolaannya harus dikuasai oleh Negara untuk memberi nilai tambah secara nyata bagi perekonomian nasional dalam usaha mencapai kemakmuran dan kesejahteraan rakyat secara berkeadilan; b.  bahwa kegiatan usaha   pertambangan mineral dan batubara yang merupakan kegiatan usaha pertambangan di luar panas bumi, minyak dan gas bumi serta air tanah mempunyai peranan penting dalam memberikan nilai tambah secara nyata kepada pertumbuhan ekonomi nasional dan pembangunan daerah secara berkelanjutan; c.  bahwa dengan mempertimbangkan perkembangan n

In [17]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/28/08/9dcdaa5aac4634e4c23af26d92121f7ce445c630efa0d3037881ae2407fb/joblib-1.3.1-py3-none-any.whl.metadata
Using cached joblib-1.3.1-py3-none-any.whl (301 kB)


In [18]:
import nltk
nltk.download('punkt')  # Download the required tokenizer data
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(text)
print(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


['d. bahwa .', '.', '.', 'UNDANG-UNDANG REPUBLIK INDONESIA  NOMOR  4 TAHUN 2009      TENTANG  PERTAMBANGAN MINERAL DAN BATUBARA   DENGAN RAHMAT TUHAN YANG MAHA ESA     PRESIDEN REPUBLIK INDONESIA,   Menimbang :  a. bahwa  mineral  dan  batubara  yang  te rkandung dalam  wilayah hukum pertambangan Indonesia merupakan kekayaan alam tak terbarukan sebagai karunia Tuhan Yang Maha Esa yang mempunyai peranan penting dalam memenuhi hajat hidup orang banyak, karena itu pengelolaannya harus dikuasai oleh Negara untuk memberi nilai tambah secara nyata bagi perekonomian nasional dalam usaha mencapai kemakmuran dan kesejahteraan rakyat secara berkeadilan; b.  bahwa kegiatan usaha   pertambangan mineral dan batubara yang merupakan kegiatan usaha pertambangan di luar panas bumi, minyak dan gas bumi serta air tanah mempunyai peranan penting dalam memberikan nilai tambah secara nyata kepada pertumbuhan ekonomi nasional dan pembangunan daerah secara berkelanjutan; c.  bahwa dengan mempertimbangkan perk

In [19]:
removal_index = []
for i, sentence in enumerate(sentences):
    if len(sentence)<30:
        removal_index.append(i)

In [20]:
removal_index = sorted(removal_index, reverse=True)

In [21]:
for idx in removal_index:
   # checking whether the corresponding iterator index is less than the list length
   if idx < len(sentences):
      # removing element by index using pop() function
      sentences.pop(idx)

sentences

['UNDANG-UNDANG REPUBLIK INDONESIA  NOMOR  4 TAHUN 2009      TENTANG  PERTAMBANGAN MINERAL DAN BATUBARA   DENGAN RAHMAT TUHAN YANG MAHA ESA     PRESIDEN REPUBLIK INDONESIA,   Menimbang :  a. bahwa  mineral  dan  batubara  yang  te rkandung dalam  wilayah hukum pertambangan Indonesia merupakan kekayaan alam tak terbarukan sebagai karunia Tuhan Yang Maha Esa yang mempunyai peranan penting dalam memenuhi hajat hidup orang banyak, karena itu pengelolaannya harus dikuasai oleh Negara untuk memberi nilai tambah secara nyata bagi perekonomian nasional dalam usaha mencapai kemakmuran dan kesejahteraan rakyat secara berkeadilan; b.  bahwa kegiatan usaha   pertambangan mineral dan batubara yang merupakan kegiatan usaha pertambangan di luar panas bumi, minyak dan gas bumi serta air tanah mempunyai peranan penting dalam memberikan nilai tambah secara nyata kepada pertumbuhan ekonomi nasional dan pembangunan daerah secara berkelanjutan; c.  bahwa dengan mempertimbangkan perkembangan nasional maupun

In [27]:
PASAL = []
for i, sentence in enumerate(sentences):
    pasal = re.search(r'Pasal[ \d]*', sentence)
    if pasal != None:
        PASAL.append(pasal.group(0))
    elif i>0:
        PASAL.append(PASAL[i-1])
    else:
        PASAL.append(pasal)

In [30]:
PASAL

[None,
 'Pasal 5 ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal  1  ',
 'Pasal 2  ',
 'Pasal 3  ',
 'Pasal 3  ',
 'Pasal  4  ',
 'Pasal  4  ',
 'Pasal 5  ',
 'Pasal 5  ',
 'Pasal 5  ',
 'Pasal 5  ',
 'Pasal 5  ',
 'Pasal  6  ',
 'Pasal  6  ',
 'Pasal  6  ',
 'Pasal   7  ',
 'Pasal   7  ',
 'Pasal   7  ',
 'Pasal 8   ',
 'Pasal 8   ',
 'Pasal 9  ',
 'Pasal 9  ',
 'Pasal  10   ',
 'Pasal 11  ',
 'Pasal 12  ',
 'Pasal  13  ',
 'Pasal  14  ',
 'Pasal  14  ',
 'Pasal